<h1> THE BATTLE OF NEIGHBORHOODS </h1>

<h4>Data Frame creation </h4>




In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_7e98299e8bb44b03bfd2a9f1e6614524 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='cC-pei4u3wF1UTKj8nMxv_qmOClxrGMfYbSMBGxQW8h_',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_7e98299e8bb44b03bfd2a9f1e6614524.get_object(Bucket='courseracapstone-donotdelete-pr-umjd8nonkkpm7w',Key='MADRID_DATA.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_mad = pd.read_excel(body)
df_mad.head()


,Hood Number,Hood,Neighborhood,Latitude,Longitude,Population,Income
0,2,ARGANZUELA,PALOS DE MOGUER,40.403638,-3.695289,40000.769957,17738
1,2,ARGANZUELA,DELICIAS,40.397292,-3.689495,40000.769957,17738
2,2,ARGANZUELA,CHOPERA,40.394893,-3.699705,40000.769957,17738
3,2,ARGANZUELA,IMPERIAL,40.406929,-3.717321,40000.769957,17738
4,2,ARGANZUELA,ATOCHA,40.405204,-3.687930,40000.769957,17738


In [2]:
df_mad.shape

(119, 7)

<h3> Explore and cluster the neighborhoods in Toronto </h3>
<h4> Obtain neighborhoods in Toronto </h4>

<h4> Foursquare initialization and libraries import </h4>

In [3]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import json # library to handle JSON files

CLIENT_ID = '0IUAI3JN1HRALCZLQKTB0F521CQWLHPQFF5HDEPTYWOZO0T2' # your Foursquare ID
CLIENT_SECRET = 'WGBZPE5050NYRY431EZOEID4GAPLX02MNWMZC054VITJQGPH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

<h4> getNearbyVenues() function definition </h4>

In [4]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    empty_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        if not results:
            empty_list.append(name)
        else:
        # return only relevant information for each nearby venue
            venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
        
    print('Empty_list:',empty_list)
    
    return(nearby_venues)

<h4> Get venues in Madrid neigborhoods </h4>

In [5]:
mad_venues = getNearbyVenues(names=df_mad['Neighborhood'],
                                   latitudes=df_mad['Latitude'],
                                   longitudes=df_mad['Longitude']
                                  )


PALOS DE MOGUER     
DELICIAS            
CHOPERA             
IMPERIAL            
ATOCHA              
ACACIAS             
LEGAZPI             
ALAMEDA DE OSUNA    
TIMON               
COMILLAS            
VISTA ALEGRE        
OPAÑEL              
PUERTA BONITA       
ABRANTES            
BUENAVISTA          
PALACIO             
EMBAJADORES         
UNIVERSIDAD         
JUSTICIA            
SOL                 
CORTES              
NUEVA ESPANA        
CIUDAD JARDIN       
PROSPERIDAD         
CASTILLA            
HISPANOAMERICA      
EL VISO             
RIOS ROSAS          
GAZTAMBIDE          
ARAPILES            
VALLEHERMOSO        
TRAFALGAR           
ALMAGRO             
COSTILLARES         
ATALAYA             
SAN PASCUAL         
COLINA              
SAN JUAN BAUTISTA   
CONCEPCION          
QUINTANA            
PUEBLO NUEVO        
VENTAS              
LA PAZ              
FUENTELARREINA      
EL PARDO            
VALVERDE            
MIRASIERRA          
EL PILAR     

<h4> Check how many venues were returned for each neighborhood </h4>

In [6]:
mad_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ABRANTES,8,8,8,8,8,8
ACACIAS,41,41,41,41,41,41
ADELFAS,41,41,41,41,41,41
ALAMEDA DE OSUNA,25,25,25,25,25,25
ALMAGRO,100,100,100,100,100,100
ALMENARA,9,9,9,9,9,9
ALMENDRALES,18,18,18,18,18,18
ALUCHE,10,10,10,10,10,10
AMPOSTA,6,6,6,6,6,6


<h4> Filter venues to obtain gyms </h4>

In [7]:
mad_gyms = mad_venues[mad_venues['Venue Category'].str.contains('Gym')]

<h4> Analyze types of gym per neighboorhood </h4>

In [8]:
# one hot encoding
mad_onehot = pd.get_dummies(mad_gyms[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mad_onehot['Neighborhood'] = mad_gyms['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mad_onehot.columns[-1]] + list(mad_onehot.columns[:-1])
mad_onehot = mad_onehot[fixed_columns]

mad_onehot.head()

,Neighborhood,Boxing Gym,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym
3,PALOS DE MOGUER,0,1,0,0,0
23,PALOS DE MOGUER,0,0,1,0,0
78,PALOS DE MOGUER,0,0,1,0,0
109,DELICIAS,0,0,1,0,0
125,CHOPERA,0,1,0,0,0


<h4> Group gyms by neighborhood </h4>

In [9]:
mad_gyms_grouped = mad_gyms.groupby('Neighborhood', as_index=False).count()
#mad_gyms_grouped['Neighborhood'] = mad_gyms['Neighborhood']
mad_gyms_grouped.rename(columns={"Neighborhood Latitude": "Gyms"}, inplace = True)
mad_gyms_grouped.drop(['Neighborhood Longitude', 'Venue','Venue Latitude','Venue Longitude','Venue Category'], axis = 1,inplace = True) 

<h4> Prepare dataframe for classification </h4>

In [10]:
#unir con dataframe original para sacar los barrios sin gym
mad_merged = df_mad

#Borrar si uso onehot
mad_grouped = mad_gyms_grouped

mad_merged = mad_merged.join(mad_grouped.set_index('Neighborhood'), on='Neighborhood')

#Replace NaN with 0
mad_merged.fillna(0,inplace=True)

mad_gr_clas = mad_merged.drop(['Hood Number', 'Hood','Neighborhood','Latitude','Longitude'], axis = 1) 
neighborhoods_venues_sorted = mad_merged.drop(['Hood Number', 'Hood','Population','Latitude','Longitude','Income'], axis = 1) 

<h4> Normalize data </h4>

In [11]:
from sklearn import preprocessing
mad_gr_clas_norm = preprocessing.StandardScaler().fit(mad_gr_clas).transform(mad_gr_clas.astype(float))

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


<h4> Cluster Neighborhoods </h4>

In [12]:
# set number of clusters
kclusters = 4

#mad_grouped_clustering = mad_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mad_gr_clas_norm)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mad_merged = df_mad

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mad_merged = mad_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mad_merged.head() # check the last columns!


,Hood Number,Hood,Neighborhood,Latitude,Longitude,Population,Income,Cluster Labels,Gyms
0,2,ARGANZUELA,PALOS DE MOGUER,40.403638,-3.695289,40000.769957,17738,0,3.0
1,2,ARGANZUELA,DELICIAS,40.397292,-3.689495,40000.769957,17738,1,1.0
2,2,ARGANZUELA,CHOPERA,40.394893,-3.699705,40000.769957,17738,1,1.0
3,2,ARGANZUELA,IMPERIAL,40.406929,-3.717321,40000.769957,17738,0,3.0
4,2,ARGANZUELA,ATOCHA,40.405204,-3.687930,40000.769957,17738,1,1.0


<h4> Get geographical coordinates of Madrid </h4>

In [13]:
address = 'Madrid'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid are 40.4167047, -3.7035825.


<h4> Visualize resulting clusters </h4>

In [14]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mad_merged['Latitude'], mad_merged['Longitude'], mad_merged['Neighborhood'], mad_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h4> Cluster 1 </h4>

In [15]:
cluster1 = mad_merged.loc[mad_merged['Cluster Labels'] == 0]
cluster1.describe()

,Hood Number,Latitude,Longitude,Population,Income,Cluster Labels,Gyms
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.000000
mean,6.105263,40.431660,-3.685068,39800.165048,19765.789474,0.0,2.526316
std,5.248224,0.025271,0.022557,6139.909612,3905.253165,0.0,0.611775
min,1.000000,40.401396,-3.717321,26465.798055,15111.000000,0.0,2.000000
25%,2.500000,40.408305,-3.703299,38235.683804,16711.000000,0.0,2.000000
50%,4.000000,40.429397,-3.684004,40000.769957,18620.000000,0.0,2.000000
75%,6.500000,40.446113,-3.670395,43906.854995,23790.000000,0.0,3.000000
max,16.000000,40.483264,-3.641543,48614.187994,26267.000000,0.0,4.000000


<h4> Cluster 2 </h4>

In [16]:
cluster2 = mad_merged.loc[mad_merged['Cluster Labels'] == 1]
cluster2.describe()

,Hood Number,Latitude,Longitude,Population,Income,Cluster Labels,Gyms
count,30.000000,30.000000,30.000000,30.000000,30.0000,30.0,30.000000
mean,6.866667,40.437845,-3.691982,34278.453826,22125.4000,1.0,0.533333
std,4.352830,0.025052,0.037778,5610.801597,2826.9082,0.0,0.507416
min,2.000000,40.391172,-3.787093,26465.798055,17738.0000,1.0,0.000000
25%,3.250000,40.420231,-3.706253,27645.881460,19364.5000,1.0,0.000000
50%,6.000000,40.439519,-3.688712,35602.271233,22792.0000,1.0,1.000000
75%,9.000000,40.455753,-3.673825,39559.498419,22897.0000,1.0,1.000000
max,16.000000,40.490520,-3.614116,41029.493508,26267.0000,1.0,1.000000


<h4> Cluster 3 </h4>

In [17]:
cluster3 = mad_merged.loc[mad_merged['Cluster Labels'] == 2]
cluster3.describe()

,Hood Number,Latitude,Longitude,Population,Income,Cluster Labels,Gyms
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.0,32.000000
mean,16.593750,40.399179,-3.646680,27967.352552,12353.468750,2.0,0.250000
std,3.320118,0.031243,0.044603,8337.030450,2343.732025,0.0,0.508001
min,12.000000,40.339934,-3.715212,10706.456326,9552.000000,2.0,0.000000
25%,14.000000,40.377513,-3.691244,19841.837538,9875.000000,2.0,0.000000
50%,17.000000,40.400824,-3.636445,33422.257189,12836.500000,2.0,0.000000
75%,20.000000,40.425977,-3.615950,34211.558225,13703.000000,2.0,0.000000
max,21.000000,40.457581,-3.569656,34643.006157,17807.000000,2.0,2.000000


<h4> Cluster 4 </h4>

In [18]:
cluster4 = mad_merged.loc[mad_merged['Cluster Labels'] == 3]
cluster4.describe()

,Hood Number,Latitude,Longitude,Population,Income,Cluster Labels,Gyms
count,38.000000,38.000000,38.000000,38.000000,38.000000,38.0,38.000000
mean,9.921053,40.428278,-3.701972,52296.949100,14073.736842,3.0,0.368421
std,4.141951,0.049327,0.041693,4715.197813,3172.650641,0.0,0.541322
min,1.000000,40.300496,-3.792457,43906.854995,9706.000000,3.0,0.000000
25%,8.000000,40.388139,-3.727782,48614.187994,10988.000000,3.0,0.000000
50%,10.500000,40.424683,-3.705286,53260.451337,15111.000000,3.0,0.000000
75%,13.000000,40.462365,-3.663605,56883.459157,16711.000000,3.0,1.000000
max,15.000000,40.557883,-3.634473,58297.941619,18661.000000,3.0,2.000000


In [19]:
cluster1.sort_values(by=['Population', 'Income'])

,Hood Number,Hood,Neighborhood,Latitude,Longitude,Population,Income,Cluster Labels,Gyms
81,3,RETIRO,ADELFAS,40.401903,-3.670958,26465.798055,21598,0,2.0
82,3,RETIRO,PACIFICO,40.401396,-3.674883,26465.798055,21598,0,3.0
23,5,CHAMARTIN,PROSPERIDAD,40.445414,-3.666558,33535.262135,26267,0,3.0
28,7,CHAMBERI,GAZTAMBIDE,40.434680,-3.714903,37669.280331,22897,0,2.0
84,4,SALAMANCA,CASTELLANA,40.433823,-3.684004,38235.683804,24683,0,2.0
85,4,SALAMANCA,FUENTE DEL BERRO,40.425131,-3.664238,38235.683804,24683,0,4.0
88,4,SALAMANCA,LISTA,40.429397,-3.675399,38235.683804,24683,0,3.0
89,4,SALAMANCA,GOYA,40.424816,-3.675843,38235.683804,24683,0,2.0
0,2,ARGANZUELA,PALOS DE MOGUER,40.403638,-3.695289,40000.769957,17738,0,3.0
3,2,ARGANZUELA,IMPERIAL,40.406929,-3.717321,40000.769957,17738,0,3.0
